In [1]:
import requests
from urllib.parse import urlencode, urlparse, parse_qs
import webbrowser
import time

# Replace with your Authentik configuration
AUTHENTIK_CONFIG = {
    "issuer": "http://163.114.159.68:8001/application/o/open-webui/",
    "authorization_endpoint": "http://163.114.159.68:8001/application/o/authorize/",
    "token_endpoint": "http://163.114.159.68:8001/application/o/token/",
    "userinfo_endpoint": "http://163.114.159.68:8001/application/o/userinfo/",
    "client_id": "rVzfdONVVWKEw3T601zp9sdyAVa6l4XUtTRh7bH9",          # Replace with your client ID
    "client_secret": "Zaxc9q7n8RCsJgNy728AuxRqpyAYsobbfukHIaNOUVa4EyntM10KfdpqyYVx1ZMeboEyXpjiYG72zX3erG2WczFWytle8ssZlh1ox9JnM1in7mmfKMHAaePZASc30qLB",  # Replace with your client secret
    "redirect_uri": "http://163.114.159.68:8080/oauth/oidc/callback",  # Your redirect URI
    "scope": "openid email profile",  # Requested scopes
}

# Step 1: Generate the Authorization URL
def generate_auth_url():
    params = {
        "client_id": AUTHENTIK_CONFIG["client_id"],
        "redirect_uri": AUTHENTIK_CONFIG["redirect_uri"],
        "response_type": "code",  # Authorization Code Grant
        "scope": AUTHENTIK_CONFIG["scope"],
        "state": "test_state",  # Random state for CSRF protection
    }
    url = f"{AUTHENTIK_CONFIG['authorization_endpoint']}?{urlencode(params)}"
    return url

# Step 2: Simulate the User Authorization Flow
def get_auth_code():
    auth_url = generate_auth_url()
    print(f"Opening the authorization URL in your browser: {auth_url}")
    time.sleep(1)  # To make the print visible
    webbrowser.open(auth_url)
    print("After you approve the app, paste the redirect URL here:")
    redirect_response = input("Redirect URL: ").strip()
    print(f"Received redirect response: {redirect_response}")
    # Extract the "code" and "state" from the redirect URL
    parsed_url = urlparse(redirect_response)
    query_params = parse_qs(parsed_url.query)
    auth_code = query_params.get("code", [None])[0]
    state = query_params.get("state", [None])[0]
    
    if not auth_code or not state:
        raise Exception("Failed to get authorization code or state.")
    
    print(f"Received Authorization Code: {auth_code}")
    return auth_code

# Step 3: Exchange Authorization Code for Tokens
def exchange_code_for_tokens(auth_code):
    token_endpoint = AUTHENTIK_CONFIG["token_endpoint"]
    data = {
        "grant_type": "authorization_code",
        "code": auth_code,
        "redirect_uri": AUTHENTIK_CONFIG["redirect_uri"],
        "client_id": AUTHENTIK_CONFIG["client_id"],
        "client_secret": AUTHENTIK_CONFIG["client_secret"],
    }
    response = requests.post(token_endpoint, data=data)
    if response.status_code != 200:
        raise Exception(f"Failed to retrieve tokens: {response.text}")
    
    tokens = response.json()
    print(f"Access Token: {tokens.get('access_token')}")
    print(f"ID Token: {tokens.get('id_token')}")
    print(f"Refresh Token: {tokens.get('refresh_token')}")
    return tokens

# Step 4: Retrieve UserInfo using Access Token
def get_userinfo(access_token):
    userinfo_endpoint = AUTHENTIK_CONFIG["userinfo_endpoint"]
    headers = {"Authorization": f"Bearer {access_token}"}
    response = requests.get(userinfo_endpoint, headers=headers)
    if response.status_code != 200:
        raise Exception(f"Failed to retrieve userinfo: {response.text}")
    
    userinfo = response.json()
    print(f"UserInfo: {userinfo}")
    return userinfo


# Step 1: User Authorization
auth_code = get_auth_code()

# Step 2: Exchange Authorization Code for Tokens
tokens = exchange_code_for_tokens(auth_code)
access_token = tokens.get("access_token")

# Step 3: Retrieve UserInfo
if access_token:
    get_userinfo(access_token)
else:
    print("Access token not found.")



Opening the authorization URL in your browser: http://163.114.159.68:8001/application/o/authorize/?client_id=rVzfdONVVWKEw3T601zp9sdyAVa6l4XUtTRh7bH9&redirect_uri=http%3A%2F%2F163.114.159.68%3A8080%2Foauth%2Foidc%2Fcallback&response_type=code&scope=openid+email+profile&state=test_state
After you approve the app, paste the redirect URL here:
Received redirect response: http://163.114.159.68:8080/oauth/oidc/callback?code=0912377efc1c47279e6929b2cd09672c&state=test_state
Received Authorization Code: 0912377efc1c47279e6929b2cd09672c
Access Token: eyJhbGciOiJSU0EtT0FFUC0yNTYiLCJlbmMiOiJBMjU2Q0JDLUhTNTEyIiwia2lkIjoiMTAzZTQyNzUwNjc5NzdhY2VmZDQ5ZmM1YWFhMTEyMmUiLCJ0eXAiOiJKV0UifQ.oLM-rRur6HDbdYHYXHI6YG2l_GTSkMSjOR4mm9f5GPZxoHl8rLXFzKD4hDVtClF2yzjrjjFm6fyS4dr-CjnXRfrdyB64vIsOYweNvDwRlYF7wyqkE_SByp06w2lus7lM0uiw3dE3Y4zfdg-nyOrtIYxVeODASCYqFbuMfI7LnFc6PCBtBL712d_kvLWIKOnAu46HnbCOkQcBKELGKgTpAyfJdScyZcEiRSMGDbi-xbg8lPvsrmB_XtIvRpsSMBQGNp3YxEktPCGpBRwYA2PMq60cavPGWxlWbhDCupmlJUO-NUyREu-O3nIFa0ifsOuM

In [3]:
tokens

{'access_token': 'eyJhbGciOiJSU0EtT0FFUC0yNTYiLCJlbmMiOiJBMjU2Q0JDLUhTNTEyIiwia2lkIjoiMTAzZTQyNzUwNjc5NzdhY2VmZDQ5ZmM1YWFhMTEyMmUiLCJ0eXAiOiJKV0UifQ.oLM-rRur6HDbdYHYXHI6YG2l_GTSkMSjOR4mm9f5GPZxoHl8rLXFzKD4hDVtClF2yzjrjjFm6fyS4dr-CjnXRfrdyB64vIsOYweNvDwRlYF7wyqkE_SByp06w2lus7lM0uiw3dE3Y4zfdg-nyOrtIYxVeODASCYqFbuMfI7LnFc6PCBtBL712d_kvLWIKOnAu46HnbCOkQcBKELGKgTpAyfJdScyZcEiRSMGDbi-xbg8lPvsrmB_XtIvRpsSMBQGNp3YxEktPCGpBRwYA2PMq60cavPGWxlWbhDCupmlJUO-NUyREu-O3nIFa0ifsOuMeQnoQ2Ohh2Df72IrkG2R_VtHl5-NxlswraytcozeEojx1lNKqQVRonMxDPbjU-jMHUakbFpMUql3nGBDM1vCxE3QyMapOSc--NFUr3-0J0hzL7uwjEsg7H96KKQ0mUaahYQefNo_g9SwRp5lTC45bz5XnjVcc02hb1UlXywt8qWlCQFQceO_4FMG_6DH8N3jO-zw8VVz92ox61LfPu5pFfSEHu_Kcd8B3mVJku7a4NPTKqyuhOxi0aS5IEexV7j5eMupXZC32Z8mQbTg6IOvjZmvi_-ZO1LrGQU5o-WDdOaLFcico4iZegbkmPruME7ucLZNXNCzL5f1UdjDMn__jA0eECuXrmuPOfteDesAmog.34lvq6jZnNWlLDRwX8hAHQ.J0Arqo9vNiwCI5zBbkhXeGJdeuFe2eaqvbemV0pakK5YNTZiBfah22z9Ha8zdiwV0WMu-NX43sVaLDTWI7eiD3Q1dwT3ZO7idTfNjq624zBW3DjfXxCey2RNPlamZfYdHUgeetykxlaKgncE

In [1]:
from authlib.integrations.starlette_client import OAuth
OAUTH_PROVIDERS= {'oidc': {'client_id': 'rVzfdONVVWKEw3T601zp9sdyAVa6l4XUtTRh7bH9', 'client_secret': 'Zaxc9q7n8RCsJgNy728AuxRqpyAYsobbfukHIaNOUVa4EyntM10KfdpqyYVx1ZMeboEyXpjiYG72zX3erG2WczFWytle8ssZlh1ox9JnM1in7mmfKMHAaePZASc30qLB', 'server_metadata_url': 'http://163.114.159.68:8001/application/o/open-webui/.well-known/openid-configuration', 'scope': 'openid email profile', 'name': 'authentik', 'redirect_uri': 'http://163.114.159.68:8080/oauth/oidc/callback'}}

In [2]:
oauth = OAuth()
for provider_name, provider_config in OAUTH_PROVIDERS.items():
    oauth.register(
        name=provider_name,
        client_id=provider_config["client_id"],
        client_secret=provider_config["client_secret"],
        server_metadata_url=provider_config["server_metadata_url"],
        client_kwargs={
            "scope": provider_config["scope"],
        },
        redirect_uri=provider_config["redirect_uri"],
    )

In [3]:
client = oauth.create_client('oidc')


In [4]:
from fastapi import Request

In [5]:
class d:
    def __init__ (self,query_params):
        self.query_params = query_params
        self.session={}
        self.redirect_uri=""

In [8]:
r = {'code': '4659985056fd4a05a96376c51481e81b', 'state': 'vFVwjKRQdbyRBB6BrkZITQIGmUCOY3'}
request = d(r)

In [9]:
await client.authorize_access_token(request=request)

OAuthError: invalid_grant: The provided authorization grant or refresh token is invalid, expired, revoked, does not match the redirection URI used in the authorization request, or was issued to another client

In [80]:
import stripe

In [12]:
import requests

url = "http://localhost:4040/key/info"
key = "sk-VTukT47OBfIoc7xuogx-hg"
auth_token = "sk-1234"

# Constructing the headers
headers = {
    "Authorization": f"Bearer {auth_token}"
}

# Sending the GET request
response = requests.get(url, params={"key": key}, headers=headers)

# Printing the response
print(response.status_code)
print(response.json())

200
{'key': 'sk-VTukT47OBfIoc7xuogx-hg', 'info': {'key_name': 'sk-...x-hg', 'key_alias': 'test-key', 'soft_budget_cooldown': False, 'spend': 9.876271, 'expires': None, 'models': ['all-team-models'], 'aliases': {}, 'config': {}, 'user_id': 'user-15', 'team_id': None, 'permissions': {}, 'max_parallel_requests': None, 'metadata': {}, 'blocked': None, 'tpm_limit': None, 'rpm_limit': None, 'max_budget': 10.0, 'budget_duration': None, 'budget_reset_at': None, 'allowed_cache_controls': [], 'model_spend': {}, 'model_max_budget': {}, 'budget_id': None, 'created_at': '2024-12-11T08:57:17.779000+00:00', 'updated_at': '2024-12-13T10:02:49.005000+00:00', 'litellm_budget_table': None}}


In [13]:
response.json()

{'key': 'sk-VTukT47OBfIoc7xuogx-hg',
 'info': {'key_name': 'sk-...x-hg',
  'key_alias': 'test-key',
  'soft_budget_cooldown': False,
  'spend': 9.876271,
  'expires': None,
  'models': ['all-team-models'],
  'aliases': {},
  'config': {},
  'user_id': 'user-15',
  'team_id': None,
  'permissions': {},
  'max_parallel_requests': None,
  'metadata': {},
  'blocked': None,
  'tpm_limit': None,
  'rpm_limit': None,
  'max_budget': 10.0,
  'budget_duration': None,
  'budget_reset_at': None,
  'allowed_cache_controls': [],
  'model_spend': {},
  'model_max_budget': {},
  'budget_id': None,
  'created_at': '2024-12-11T08:57:17.779000+00:00',
  'updated_at': '2024-12-13T10:02:49.005000+00:00',
  'litellm_budget_table': None}}

In [3]:
import requests
import json

url = "http://localhost:4040/key/update"
auth_token = "sk-1234"

# Constructing headers
headers = {
    "Authorization": f"Bearer {auth_token}",
    "Content-Type": "application/json"
}

# Data to be sent in the POST request
data = {
    "key": "sk-VTukT47OBfIoc7xuogx-hg",
    "user_id": "user-15",
    "max_budget": 10,
    "spend" : 9
}

# Sending the POST request
response = requests.post(url, headers=headers, data=json.dumps(data))

# Printing the response
print(response.status_code)
print(response.json())


200
{'key': 'sk-VTukT47OBfIoc7xuogx-hg', 'token': 'c9f39d62231e157a4dc466d0fee0695860ab364cd52e979e7707306db1b0fb85', 'key_name': 'sk-...x-hg', 'key_alias': 'test-key', 'soft_budget_cooldown': False, 'spend': 9.0, 'expires': None, 'models': ['all-team-models'], 'aliases': {}, 'config': {}, 'user_id': 'user-15', 'team_id': None, 'permissions': {}, 'max_parallel_requests': None, 'metadata': {}, 'blocked': None, 'tpm_limit': None, 'rpm_limit': None, 'max_budget': 10.0, 'budget_duration': None, 'budget_reset_at': None, 'allowed_cache_controls': [], 'model_spend': {}, 'model_max_budget': {}, 'budget_id': None, 'created_at': '2024-12-11T08:57:17.779000+00:00', 'updated_at': '2024-12-13T10:00:29.291000+00:00', 'litellm_budget_table': None}


In [77]:
import requests

url = 'http://localhost:8000/stripe/webhook'


headers = {
    'Content-Type': 'application/json'
}

response = requests.post(url, json=payload, headers=headers)
print(response.status_code)
print(response.json())

400
{'detail': 'Invalid signature'}


In [68]:
url = 'http://localhost:8000/stripe/test'
response = requests.get(url)
response.content

b'{"message":"Hello"}'

In [65]:
print(response.text)

<!doctype html>
<html lang="en">
	<head>
		<meta charset="utf-8" />
		<link rel="icon" type="image/png" href="/favicon/favicon-96x96.png" sizes="96x96" />
		<link rel="icon" type="image/svg+xml" href="/favicon/favicon.svg" />
		<link rel="shortcut icon" href="/favicon/favicon.ico" />
		<link rel="apple-touch-icon" sizes="180x180" href="/favicon/apple-touch-icon.png" />
		<meta name="apple-mobile-web-app-title" content="Open WebUI" />
		<link rel="manifest" href="/favicon/site.webmanifest" />
		<meta
			name="viewport"
			content="width=device-width, initial-scale=1, maximum-scale=1, viewport-fit=cover"
		/>
		<meta name="theme-color" content="#171717" />
		<meta name="robots" content="noindex,nofollow" />
		<meta name="description" content="Open WebUI" />
		<link
			rel="search"
			type="application/opensearchdescription+xml"
			title="Open WebUI"
			href="/opensearch.xml"
		/>

		<script>
			function resizeIframe(obj) {
				obj.style.height = obj.contentWindow.document.documentElement

In [2]:
import openai

api_key = "sk-VTukT47OBfIoc7xuogx-hg"
client = openai.OpenAI(
    api_key=api_key,  # Use your LiteLLM proxy key if using virtual keys
    base_url="http://0.0.0.0:4040"  # LiteLLM proxy base URL,
)

# Llama

In [91]:
response = client.chat.completions.create(
    model="Meta_Llama_31_8b_it",
    messages=[
        {
            "role": "user",
            "content": "Parle moi de la seconde guerre mondiale avec un max de détails"
        }
    ]
)
print(response.choices[0].message.content)

BadRequestError: Error code: 400 - {'error': {'message': 'Budget has been exceeded! Current cost: 5.845, Max budget: 5.0', 'type': 'budget_exceeded', 'param': None, 'code': '400'}}

In [ ]:
from litellm import token_counter

messages = [{"role": "user", "content": "Hey, how's it going"}]
token_count = token_counter(model="gpt-3.5-turbo", messages=messages)
print(f"Token count: {token_count}")

# Lucie

In [20]:
response = client.chat.completions.create(
    model="Lucie_7B",
    temperature=0.6,
    user='user3@maif.com',
    messages=[
        {
            "role": "user",
            "content": "Parle moi de la révolution française"
        }
    ]
)
print(response.choices[0].message.content)

La Révolution française de 1789 fut un événement majeur de l'histoire de France et du monde. Elle a commencé avec la prise de la Bastille le 14 juillet 1789 et s'est achevée avec la proclamation de la Déclaration des droits de l'Homme et du Citoyen le 26 août 1789.

La Révolution française a eu de nombreuses causes, telles que la montée de l'absolutisme royal, la crise financière et économique, l'inégalité sociale et les tensions politiques. Elle a également été influencée par les idées des Lumières et de la philosophie radicale.

La Révolution française a entraîné de nombreux changements, notamment la fin de la monarchie absolue et l'établissement de la République. Elle a également conduit à la Déclaration des droits de l'Homme et du Citoyen, qui a établi des principes fondamentaux tels que la liberté, l'égalité et la fraternité.

Cependant, la Révolution française a également été marquée par la violence et la terreur, avec l'exécution de nombreux nobles et la mise en place de la Terr

In [ ]:
response.

ChatCompletion(id='chat-58e6116d091c4f06b7f59a6148f97225', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="La Révolution française de 1789 fut un événement majeur de l'histoire de France et du monde. Elle a commencé avec la prise de la Bastille le 14 juillet 1789 et s'est achevée avec la proclamation de la Déclaration des droits de l'Homme et du Citoyen le 26 août 1789.\n\nLa Révolution française a eu de nombreuses causes, telles que la montée de l'absolutisme royal, la crise financière et économique, l'inégalité sociale et les tensions politiques. Elle a également été influencée par les idées des Lumières et de la philosophie radicale.\n\nLa Révolution française a entraîné de nombreux changements, notamment la fin de la monarchie absolue et l'établissement de la République. Elle a également conduit à la Déclaration des droits de l'Homme et du Citoyen, qui a établi des principes fondamentaux tels que la liberté, l'égalité et la frate

In [ ]:
curl -X GET 'http://localhost:4000/global/spend/report?start_date=2024-12-01&end_date=2024-12-31&group_by=team' \
  -H 'Authorization: Bearer sk-1234'

In [22]:
import requests
url = 'http://localhost:4000/global/spend/report'
params = {
    'start_date': '2024-12-01',
    'end_date': '2024-12-30'
}

headers = {
    'Authorization': 'Bearer sk-1234'
}

# Make the GET request
response = requests.get(url, headers=headers, params=params)
spend_report = response.json()

* Get list of models

In [23]:
!curl -X 'GET' \
  'http://localhost:4000/v1/models' \
  -H 'accept: application/json' \
  -H 'Authorization: Bearer sk-1234'

{"data":[{"id":"Meta_Llama_31_8b_it","object":"model","created":1677610602,"owned_by":"openai"},{"id":"Lucie_7B","object":"model","created":1677610602,"owned_by":"openai"}],"object":"list"}

* Generate key via api

In [ ]:
!curl -X 'POST' \
  'http://localhost:4000/key/generate' \
  -H 'accept: application/json' \
  -H 'Content-Type: application/json' \
  -H 'Authorization: Bearer sk-1234' \
  -d '{
    "key_alias": "Sales",
    "models": ["Lucie_7B"],
    "max_budget": 10,
    "user_id": "user1_id",
    "team_id": "team1_id",
    "max_parallel_requests": 10,
    "blocked": true,
    "aliases": {}
  }'

IndentationError: unindent does not match any outer indentation level (<string>, line 10)

In [29]:
!curl -X 'GET' \
  'http://localhost:4000/key/list?page=1&size=10' \
  -H 'accept: application/json' \
  -H 'Authorization: Bearer sk-1234' \


{"keys":["88dc28d0f030c55ed4ab77ed8faf098196cb1c05df778539800c9f1243fe6b4b","983002003de2c682d18c86f0f450f9b3469d88021e7a8e01bb28ba46adeff48a","d1117f612d7a35a93e69584ff2d595aa603e598e980dd4d1fd9640b5f884b1f2"],"total_count":3,"current_page":1,"total_pages":1}curl: (6) Could not resolve host: \


In [4]:
!curl -X GET "http://0.0.0.0:4000/spend/tags" \
-H "Authorization: Bearer sk-1234"

[]

In [15]:
!curl -X 'GET' \
  'http://localhost:4000/global/spend/report?start_date=2024-12-02&end_date=2024-12-10&group_by=MAIF&api_key=sk-1234&internal_user_id=user1' \
  -H 'accept: application/json'

{"error":{"message":"Authentication Error, No api key passed in.","type":"auth_error","param":"None","code":"401"}}